In [1]:
# ! pip install faiss-cpu
# ! pip install sentence-transformers

In [2]:
import pandas as pd
pd.set_option('display.max_colwidth', 100)
df = pd.read_csv("sample_text.csv")
df.shape

(8, 2)

In [3]:
df

,text,category
0,Meditation and yoga can improve mental health,Health
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
2,These are the latest fashion trends for this week,Fashion
3,Vibrant color jeans for male are becoming a trend,Fashion
4,The concert starts at 7 PM tonight,Event
5,Navaratri dandiya program at Expo center in Mumbai this october,Event
6,Exciting vacation destinations for your next trip,Travel
7,Maldives and Srilanka are gaining popularity in terms of low budget vacation places,Travel


#### Step 1 : Create source embeddings for the text column

In [4]:
from sentence_transformers import SentenceTransformer
encoder = SentenceTransformer("all-mpnet-base-v2")
vectors = encoder.encode(df.text) # array of texts are inserted

d:\LLM Projects\Krish Naik Langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\LLM Projects\Krish Naik Langchain\venv\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
vectors.shape

(8, 768)

In [6]:
vectors

array([[-0.00247395,  0.03626723, -0.05290459, ..., -0.09152358,
        -0.03969997, -0.04330491],
       [-0.03357265,  0.00980516, -0.0325013 , ..., -0.05165467,
         0.02245887, -0.03156184],
       [-0.0186532 , -0.04051321, -0.01235388, ...,  0.00610587,
        -0.07179647,  0.02773852],
       ...,
       [-0.00066458,  0.04252128, -0.05645507, ...,  0.01315469,
        -0.03183567, -0.04357662],
       [-0.03317154,  0.03252459, -0.0248484 , ...,  0.01174422,
         0.05747123,  0.00571021],
       [-0.00166393,  0.00413826, -0.04597082, ...,  0.02008527,
         0.05656241, -0.00161597]], dtype=float32)

In [7]:
dim = vectors.shape[1]
dim

768

#### Step 2 : Build a FAISS Index for vectors

In [8]:
import faiss
index = faiss.IndexFlatL2(dim) 
# Similar to database index, it's just creating an index, that allows faster search
# Create an empty index of size 768

In [9]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002376B873D80> >

#### Step 3 : Normalize the source vectors (as we are using L2 distance to measure similarity) and add to the index

In [10]:
index.add(vectors) # add vectors to the empty index

In [11]:
index

<faiss.swigfaiss_avx2.IndexFlatL2; proxy of <Swig Object of type 'faiss::IndexFlatL2 *' at 0x000002376B873D80> >

#### Step 4 : Encode search text using same encorder and normalize the output vector

In [12]:
# search_query = "I want to buy a polo t-shirt"
# search_query = "looking for places to visit during the holidays"
search_query = "An apple a day keeps the doctor away"
vec = encoder.encode(search_query)
vec.shape # One dimensional

(768,)

In [13]:
vec

array([ 1.77042782e-02,  7.70549625e-02, -1.25192907e-02, -1.39331154e-03,
        5.92106348e-03, -1.42679419e-02, -4.72583845e-02,  3.91949993e-03,
        4.41600792e-02,  3.09807789e-02,  9.15983394e-02,  2.02602874e-02,
       -2.16005202e-02,  1.33208102e-02, -4.14275974e-02,  4.16395254e-02,
        4.37019430e-02,  6.06419612e-03,  2.21781209e-02, -1.94715429e-02,
       -3.44190747e-02,  3.79219614e-02,  5.27905393e-03,  2.22010948e-02,
       -1.11636277e-02, -2.53350977e-02,  7.79388398e-02,  3.47347856e-02,
        1.20056877e-02, -6.52653873e-02, -4.83940952e-02, -3.03445403e-02,
       -6.99562579e-02, -2.53461432e-02,  1.78119137e-06,  1.50222173e-02,
        2.15191599e-02,  2.21124589e-02, -8.38034600e-02,  3.70960613e-03,
       -2.21117996e-02, -8.53062123e-02,  6.44896238e-04,  1.62597857e-02,
       -1.88165624e-02,  3.99218090e-02, -3.13138738e-02,  2.47449819e-02,
       -7.98756350e-03,  6.11144789e-02, -1.47846388e-02,  4.10234279e-05,
        1.20446244e-02,  

In [14]:
import numpy as np
svec = np.array(vec).reshape(1,-1)
svec.shape # Two dimensional

(1, 768)

In [15]:
svec

array([[ 1.77042782e-02,  7.70549625e-02, -1.25192907e-02,
        -1.39331154e-03,  5.92106348e-03, -1.42679419e-02,
        -4.72583845e-02,  3.91949993e-03,  4.41600792e-02,
         3.09807789e-02,  9.15983394e-02,  2.02602874e-02,
        -2.16005202e-02,  1.33208102e-02, -4.14275974e-02,
         4.16395254e-02,  4.37019430e-02,  6.06419612e-03,
         2.21781209e-02, -1.94715429e-02, -3.44190747e-02,
         3.79219614e-02,  5.27905393e-03,  2.22010948e-02,
        -1.11636277e-02, -2.53350977e-02,  7.79388398e-02,
         3.47347856e-02,  1.20056877e-02, -6.52653873e-02,
        -4.83940952e-02, -3.03445403e-02, -6.99562579e-02,
        -2.53461432e-02,  1.78119137e-06,  1.50222173e-02,
         2.15191599e-02,  2.21124589e-02, -8.38034600e-02,
         3.70960613e-03, -2.21117996e-02, -8.53062123e-02,
         6.44896238e-04,  1.62597857e-02, -1.88165624e-02,
         3.99218090e-02, -3.13138738e-02,  2.47449819e-02,
        -7.98756350e-03,  6.11144789e-02, -1.47846388e-0

#### Step 5: Search for similar vector in the FAISS index created

In [16]:
distances, I = index.search(svec, k=2) # K  = How many similar vectors
distances # Distances

array([[1.3433158, 1.7125273]], dtype=float32)

In [17]:
I # Index in the original dataframe

array([[1, 0]], dtype=int64)

In [18]:
df.loc[I[0]]

,text,category
1,"Fruits, whole grains and vegetables helps control blood pressure",Health
0,Meditation and yoga can improve mental health,Health


In [19]:
search_query

'An apple a day keeps the doctor away'

In [20]:
# You can see that the two results from the dataframe are similar to a search_query